In [1]:
# Desafio 3
import psycopg2
import pandas as pd
from decouple import config

In [2]:
df_itens_venda = pd.read_excel('../itens_venda.xlsx')
df_venda = pd.read_excel('../venda.xlsx')
df_cliente  = pd.read_excel('../cliente.xlsx')
df_categoria = pd.read_excel('../categoria.xlsx')
df_produto = pd.read_excel('../produto.xlsx')

In [3]:
df_produto.head()

,id,produto,preco,categoria_id
0,122,industry most,17.46,124
1,554,involve machine,19.39,124
2,936,risk care,80.14,124
3,397,least prove,29.28,842
4,322,think already,155.39,249


In [4]:
df_cliente.head()

,id,nome
0,460,Thomas Anthony
1,981,Alexander Hancock
2,909,Tony Green
3,715,Stephanie Becker
4,778,Micheal Dixon


In [5]:
def insert_cliente():
    for i, row in df_cliente.iterrows():
        name = row["nome"]
        cursor.execute(f"INSERT INTO cliente (nome) VALUES ('{name}') ON CONFLICT DO NOTHING")  # noqa
    connection.commit()

In [6]:
#Adicionando as novas tabela no banco "my_db"
try:
    connection = psycopg2.connect(user = "my_user",
                                  password = config("PASSWORD"),
                                  host = "127.0.0.1",
                                  port = "5432",
                                  database = "my_db")

    cursor = connection.cursor()
    # Print PostgreSQL Connection properties
    print ( connection.get_dsn_parameters(),"\n")

    # Print PostgreSQL version
    cursor.execute("SELECT version();")
    record = cursor.fetchone()
    print("You are connected to - ", record,"\n")

except (Exception, psycopg2.Error) as error :
    print ("Error while connecting to PostgreSQL", error)

{'user': 'my_user', 'dbname': 'my_db', 'host': '127.0.0.1', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 11.7 (Ubuntu 11.7-0ubuntu0.19.10.1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 9.2.1-9ubuntu2) 9.2.1 20191008, 64-bit',) 



In [7]:
cur = connection.cursor()

In [8]:
insert_cliente()

In [9]:
df_categoria.head()

,id,categoria
0,169,bebidas
1,249,sobremesas
2,215,cereais
3,842,frios
4,227,limpeza


In [10]:
def insert_categoria():
    for i, row in df_categoria.iterrows():
        name = row["categoria"]
        cursor.execute(f"INSERT INTO categoria (categoria) VALUES ('{name}') ON CONFLICT DO NOTHING")  # noqa
    connection.commit()

In [11]:
insert_categoria()

In [12]:
cursor.execute('SELECT * FROM categoria;')
cursor.fetchall()

[(1, 'bebidas'),
 (2, 'sobremesas'),
 (3, 'cereais'),
 (4, 'frios'),
 (5, 'limpeza'),
 (6, 'hortifruti'),
 (7, 'perfumaria'),
 (8, 'mercearia'),
 (9, 'bazar'),
 (10, 'pet shop')]

In [13]:
def get_data(items, field='categoria'):
    '''
    Lê os dados para extrair id e um outro campo como identificador único
    e monta um dicionário.
    # https://gist.github.com/rg3915/0f63ee9bde818c4a56abb110c94b855b
    '''
    my_dict = {}
    for item in items:
        my_dict[str(item['id'])] = item[field]
    return my_dict

In [14]:
# Monta uma lista de categorias.
items = df_categoria.T.apply(dict).tolist()

In [15]:
# Monta um dicionário de categorias.
dict_categoria = get_data(items)

In [16]:
dict_categoria

{'169': 'bebidas',
 '249': 'sobremesas',
 '215': 'cereais',
 '842': 'frios',
 '227': 'limpeza',
 '678': 'hortifruti',
 '124': 'perfumaria',
 '563': 'mercearia',
 '734': 'bazar',
 '484': 'pet shop'}

In [17]:
def insert_produto():
    for i, row in df_produto.iterrows():
        produto = row["produto"]
        preco = round(row['preco'] * 5.6, 2)
        print(row['categoria_id'])
        campo = dict_categoria.get(str(row['categoria_id']))
        print(campo)
        cursor.execute(f"SELECT * FROM categoria WHERE categoria='{campo}';")
        categoria = cursor.fetchall()
        print(categoria)
        print(categoria[0][0])
        categoria_id = categoria[0][0]
        cursor.execute(f"INSERT INTO produto (produto, preco, categoria_id) VALUES ('{produto}', '{preco}', '{categoria_id}') ON CONFLICT DO NOTHING")  # noqa
    connection.commit()

In [18]:
insert_produto()

124
perfumaria
[(7, 'perfumaria')]
7
124
perfumaria
[(7, 'perfumaria')]
7
124
perfumaria
[(7, 'perfumaria')]
7
842
frios
[(4, 'frios')]
4
249
sobremesas
[(2, 'sobremesas')]
2
124
perfumaria
[(7, 'perfumaria')]
7
678
hortifruti
[(6, 'hortifruti')]
6
678
hortifruti
[(6, 'hortifruti')]
6
227
limpeza
[(5, 'limpeza')]
5
734
bazar
[(9, 'bazar')]
9
678
hortifruti
[(6, 'hortifruti')]
6
734
bazar
[(9, 'bazar')]
9
734
bazar
[(9, 'bazar')]
9
249
sobremesas
[(2, 'sobremesas')]
2
124
perfumaria
[(7, 'perfumaria')]
7
563
mercearia
[(8, 'mercearia')]
8
842
frios
[(4, 'frios')]
4
227
limpeza
[(5, 'limpeza')]
5
169
bebidas
[(1, 'bebidas')]
1
215
cereais
[(3, 'cereais')]
3
215
cereais
[(3, 'cereais')]
3
124
perfumaria
[(7, 'perfumaria')]
7
227
limpeza
[(5, 'limpeza')]
5
678
hortifruti
[(6, 'hortifruti')]
6
124
perfumaria
[(7, 'perfumaria')]
7
563
mercearia
[(8, 'mercearia')]
8
678
hortifruti
[(6, 'hortifruti')]
6
678
hortifruti
[(6, 'hortifruti')]
6
169
bebidas
[(1, 'bebidas')]
1
227
limpeza
[(5, 'limpe

In [19]:
lista = [(1, 'Regis'), (2, 'João')]

In [20]:
lista[0]

(1, 'Regis')

In [21]:
lista[0][1]

'Regis'

In [22]:
lista[0][0]

1

In [23]:
# Monta uma lista de clientes.
items_cliente = df_cliente.T.apply(dict).tolist()
items_cliente[:1]

[{'id': 460, 'nome': 'Thomas Anthony'}]

In [24]:
# Monta um dicionário de clientes.
dict_cliente = get_data(items=items_cliente, field='nome')
dict_cliente

{'460': 'Thomas Anthony',
 '981': 'Lauren Garza',
 '909': 'Tony Green',
 '715': 'Stephanie Becker',
 '778': 'Micheal Dixon',
 '193': 'Tara Scott',
 '669': 'Tina Anderson',
 '333': 'Robert Hunt',
 '745': 'Michael Johnston',
 '131': 'Heather Kelly',
 '827': 'Daniel Crawford',
 '830': 'Maria Nixon',
 '124': 'Joshua Lewis',
 '344': 'Ashley Case',
 '317': 'Sean Clark',
 '145': 'Nicole Sosa',
 '377': 'Tracy Rivera',
 '762': 'Carrie Tapia',
 '597': 'Crystal Schmidt',
 '574': 'Kimberly Gibbs',
 '930': 'Cassidy Johnson',
 '239': 'Nicole Ramos',
 '781': 'Patrick Bass',
 '166': 'Dustin Lee',
 '227': 'Emily Hull',
 '175': 'Wendy Pacheco',
 '738': 'Molly Waters',
 '633': 'Eric Meyer',
 '812': 'Craig Johnson Jr.',
 '459': 'William Reid',
 '234': 'Shirley Cook',
 '184': 'Jeffrey George',
 '670': 'Nicole Wagner',
 '209': 'James Galloway',
 '563': 'Richard English',
 '977': 'Rachel Brown',
 '361': 'Colin Reynolds',
 '709': 'Jimmy Singh',
 '525': 'Tiffany Alvarado',
 '408': 'Rodney Clements',
 '643': 'D

In [25]:
def insert_venda():
    for i, row in df_venda.iterrows():
        print(row)
        data = row['data']
        campo = dict_cliente.get(str(row['cliente_id']))
        print(campo)
        cursor.execute(f"SELECT * FROM cliente WHERE nome='{campo}';")
        cliente = cursor.fetchall()
        print(cliente)
        print(cliente[0][0])
        cliente_id = cliente[0][0]
        slug = row['slug']
        cursor.execute(f"INSERT INTO venda (data, cliente_id, slug) VALUES ('{data}', '{cliente_id}', '{slug}') ON CONFLICT DO NOTHING")  # noqa
    connection.commit()
insert_venda()

id                                             490
data                                    2020-03-15
cliente_id                                     822
slug          07f8690e-d1d8-41dc-b866-cdbbbfad445e
Name: 0, dtype: object
Jill Barber
[(48, 'Jill Barber')]
48
id                                             624
data                                    2020-05-06
cliente_id                                     778
slug          39b30e02-17a8-4527-a23b-99ed78602bb4
Name: 1, dtype: object
Micheal Dixon
[(5, 'Micheal Dixon')]
5
id                                             128
data                                    2020-05-15
cliente_id                                     762
slug          6cc58638-feea-4f3c-8da6-c9611ac1de65
Name: 2, dtype: object
Carrie Tapia
[(19, 'Carrie Tapia')]
19
id                                             792
data                                    2020-10-06
cliente_id                                     153
slug          3d6c6822-7851-483f-95d4-c375abda6e51


[(50, 'Joshua Lewis')]
50
id                                             675
data                                    2020-05-10
cliente_id                                     886
slug          ee879f96-d9e5-45c6-8e84-9244420a78ef
Name: 71, dtype: object
Donna Calderon
[(45, 'Donna Calderon')]
45
id                                             432
data                                    2020-06-11
cliente_id                                     227
slug          5d510189-489b-4723-985d-a66a317191d5
Name: 72, dtype: object
Emily Hull
[(26, 'Emily Hull')]
26
id                                             607
data                                    2020-10-10
cliente_id                                     234
slug          07300899-f717-4384-a086-5ebe16fce62f
Name: 73, dtype: object
Shirley Cook
[(32, 'Shirley Cook')]
32
id                                             493
data                                    2020-04-05
cliente_id                                     822
slug          bce6b7

In [26]:
# Monta uma lista de vendas.
items_venda = df_venda.T.apply(dict).tolist()
items_venda[:2]

[{'id': 490,
  'data': '2020-03-15',
  'cliente_id': 822,
  'slug': '07f8690e-d1d8-41dc-b866-cdbbbfad445e'},
 {'id': 624,
  'data': '2020-05-06',
  'cliente_id': 778,
  'slug': '39b30e02-17a8-4527-a23b-99ed78602bb4'}]

In [27]:
# Monta um dicionário de vendas.
dict_venda = get_data(items=items_venda, field='slug')
dict_venda

{'490': '07f8690e-d1d8-41dc-b866-cdbbbfad445e',
 '624': '5c2261e9-2d9f-4040-8d74-2b458116dd16',
 '128': '6cc58638-feea-4f3c-8da6-c9611ac1de65',
 '792': '3d6c6822-7851-483f-95d4-c375abda6e51',
 '602': '36aab7e1-4c1d-4044-9977-d2d006df0f9b',
 '194': 'aa060ef8-5931-48d9-9ea4-9aa1eeebd910',
 '976': 'b08121cb-e7cd-477f-b3be-4888a869dc5a',
 '264': 'd5feb6f3-87cf-416c-8a81-b96d88a180e9',
 '821': '94a52c2a-c7bc-4abd-9556-6b92b9064129',
 '381': '48fe67f4-3581-47cd-93ad-83596e8e593e',
 '159': 'dd7f8f47-1f54-4bec-89d2-54c11f3e8e9f',
 '888': '7edafde0-90e0-40cf-a0b5-5a79ffcbf538',
 '454': '13a3d1b1-aca5-423b-87d7-3266286590b4',
 '387': 'b623f81f-f97e-4a49-b87f-e23abe855a6a',
 '651': '95fec2a2-da73-40ef-9364-41b2b0ed983b',
 '805': 'c8af36cd-33a2-4940-a114-10a4fe0fcfa0',
 '355': '0a419692-57eb-42c5-b78d-2ea84a46e8b7',
 '296': '130077fb-dca1-45e4-b519-dc5360603c1b',
 '327': 'd66aa5b6-8e7e-465e-98d9-8f0ab16792f9',
 '478': '4d707264-2216-45f0-a52f-c3273aa0add8',
 '125': '791939b3-af89-474d-9fe2-a04771a

In [28]:
# Monta uma lista de produtos.
items_produto = df_produto.T.apply(dict).tolist()
items_produto[:2]

[{'id': 122, 'produto': 'industry most', 'preco': 17.46, 'categoria_id': 124},
 {'id': 554,
  'produto': 'involve machine',
  'preco': 19.39,
  'categoria_id': 124}]

In [29]:
# Monta um dicionário de produtos.
dict_produto = get_data(items=items_produto, field='produto')
dict_produto

{'122': 'industry most',
 '554': 'involve machine',
 '936': 'risk care',
 '397': 'least prove',
 '322': 'think already',
 '272': 'sure necessary',
 '600': 'billion pretty',
 '589': 'life social',
 '732': 'central dog',
 '684': 'also evening',
 '132': 'phone keep',
 '808': 'record single',
 '839': 'me prevent',
 '183': 'campaign learn',
 '688': 'force present',
 '301': 'probably activity',
 '274': 'main benefit',
 '148': 'politics candidate',
 '990': 'stand available',
 '465': 'yard degree',
 '530': 'ago per',
 '777': 'man cover',
 '629': 'expect help',
 '957': 'cover huge',
 '731': 'laugh worker',
 '107': 'amount interesting',
 '989': 'never they',
 '763': 'else force',
 '299': 'sense thing',
 '478': 'middle sit',
 '624': 'wall provide',
 '344': 'movie air',
 '780': 'entire glass',
 '220': 'drug step',
 '634': 'walk discuss',
 '799': 'method minute',
 '873': 'need certain',
 '422': 'parent music',
 '312': 'tough return',
 '577': 'measure hear',
 '636': 'gun whose',
 '698': 'top attorne

In [30]:
def insert_itens_venda():
    for i, row in df_itens_venda.iterrows():
        quantidade = str(row['quantidade']))
        preco_venda = row['preco_venda']
        venda_id = str(row['venda_id']).replace('.0', '')
        print(venda_id)
        venda = dict_venda.get(venda_id)
        cursor.execute(f"SELECT * FROM venda WHERE slug='{venda}';")
        venda = cursor.fetchall()
        venda_id = venda[0][0]
        produto_id = str(row['produto_id']).replace('.0', '')
        print(produto_id)
        produto = dict_produto.get(produto_id)
        cursor.execute(f"SELECT * FROM produto WHERE produto='{produto}';")
        produto = cursor.fetchall()
        print(produto)
        produto_id = produto[0][0]
        cursor.execute(f"INSERT INTO itens_venda (quantidade, preco_venda, venda_id, produto_id) VALUES ('{quantidade}', '{preco_venda}', '{venda_id}', '{produto_id}') ON CONFLICT DO NOTHING")  # noqa
    connection.commit()
insert_itens_venda()

552
308
[(47, 'responsibility ground', Decimal('257.77'), 3)]


InvalidTextRepresentation: invalid input syntax for integer: "73.0"
LINE 1: ...idade, preco_venda, venda_id, produto_id) VALUES ('73.0', '4...
                                                             ^


In [ ]:
lista2 = ['A', 'B', 'C']

In [ ]:
lista2[0]

In [ ]:
lista2[1]

In [ ]:
lista2[2]

In [ ]:
lista2[3]